In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import sys
sys.path.append('../code')
import scrape, features
import dataframe_utilities as util

# Update DB & Get Today Games

In [4]:
last_day = pd.to_datetime(pd.read_csv('../data/game_summaries.csv')['date']).max()
get_day = last_day + pd.Timedelta(days=1)
get_day

Timestamp('2024-09-27 00:00:00')

In [5]:
#home and away
#game time
#pitchers


In [6]:
import datetime

# Example of creating a datetime sobject
date_obj = datetime.date.today()
while get_day.date() < date_obj:
    links = scrape.get_game_links(get_day)
    for l in links:
        scrape.process_link(l)
    get_day += + pd.Timedelta(days=1)

In [7]:
import lxml
test_df = scrape.get_today_games()

In [8]:
test_df

,away_team_abbr,home_team_abbr,time,away_pitcher,home_pitcher,date
0,CHC,MIL,1:10PM,taillja01,reaco01,2024-05-31
1,KCR,MIN,1:10PM,lynchda02,paddach01,2024-05-31
2,OAK,TBR,1:10PM,harriho03,armstsh01,2024-05-31
3,HOU,SEA,4:10PM,arrigsp01,gilbelo01,2024-05-31
4,DET,BOS,7:10PM,flaheja01,pivetni01,2024-05-31
5,ARI,NYM,7:10PM,galleza01,scottch01,2024-05-31
6,WSN,ATL,7:20PM,willitr01,kerrra01,2024-05-31
7,NYY,LAA,9:38PM,rodonca01,sandopa02,2024-05-31


# Process Stats for Today's Games

## Merge test and train dfs

In [9]:
test_df['is_night_game'] = True
test_df['time_numeric'] = pd.to_numeric(test_df['time'].str[:1], errors='coerce')

# Set 'is_night_game' to False where 'time_numeric' is less than 5 and not NaN
test_df.loc[(test_df['time_numeric'] < 5) & (test_df['time_numeric'].notna()), 'is_night_game'] = False

# Set 'is_night_game' to True where 'time_numeric' is NaN (indicating non-numeric original entries)
test_df.loc[test_df['time_numeric'].isna(), 'is_night_game'] = True
test_df['is_night_game'][test_df['time'].str[1:2].isin(['0','1'])] = True #for 10,11 PM games
test_df.drop(columns='time', inplace=True)

In [10]:
test_df['is_test'] = True
test_df['home_team_win']=np.nan
test_df['game_id'] = test_df.home_team_abbr + test_df.date.astype('str').str.replace('-','') + '0'
test_df.shape

(8, 10)

In [11]:
df = features.get_game_df()
df['is_test'] = False
df.shape

(935, 9)

In [12]:
df = pd.concat([df,test_df])
df = df.sort_values(by='date').reset_index(drop=True)
df.shape

(943, 10)

In [13]:
df.columns

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'is_test',
       'time_numeric'],
      dtype='object')

## Add Features

In [14]:
df = features.add_trueskill_ratings(df)
df = features.add_rest_durations(df)
df.shape

(943, 19)

In [15]:
df.drop(columns=["is_test", "time_numeric"], inplace=True)

In [16]:
#datetime
date = pd.to_datetime(df['date'])
df['season'] = date.dt.year
df['month']=date.dt.month
df['week_num'] = date.dt.isocalendar().week
df['dow']=date.dt.weekday.astype('int')

In [17]:
df.shape

(943, 21)

In [18]:
df['dh_game_no'] = pd.to_numeric(df['game_id'].str[-1:],errors='coerce')
df['date'] = (pd.to_datetime(df['date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') #epoch time

In [19]:
test_df['game_id']

0    MIL202405310
1    MIN202405310
2    TBR202405310
3    SEA202405310
4    BOS202405310
5    NYM202405310
6    ATL202405310
7    LAA202405310
Name: game_id, dtype: object

### Add Stats

In [20]:
games = features.get_games()
batting = features.get_batting()
pitching = features.get_pitching()
pitchers = features.get_pitchers()
print(df.columns)

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'home_trueskill_pre',
       'away_trueskill_pre', 'ts_diff', 'home_team_rest', 'away_team_rest',
       'home_pitcher_rest', 'away_pitcher_rest', 'team_rest_diff',
       'pitcher_rest_diff', 'season', 'month', 'week_num', 'dow',
       'dh_game_no'],
      dtype='object')


#### Rolling 10 Day Stats

In [21]:
b_stats = ['batting_avg','leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging']
df = features.add_10RA_rolling(batting, df, b_stats, True,'batting')
print(df.columns)

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
938     MIL202405310            MIL            CHC  1717113600          False   
939  ATLCWS20240627D            CWS            ATL  1719446400           True   
940  CLEMIN20240809D            MIN            CLE  1723161600           True   
941  MILCIN20240830D            CIN            MIL  1724976000           True   
942  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [22]:
pitching['SO_batters_faced'] = pitching['so'] / pitching['batters_faced']
pitching['H_batters_faced'] = pitching['h'] / pitching['batters_faced']
pitching['BB_batters_faced'] = pitching['bb'] / pitching['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitching, df, b_stats, True, 'team_pitching')

AAA
AAA
AAA
AAA
             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
938     MIL202405310            MIL            CHC  1717113600          False   
939  ATLCWS20240627D            CWS            ATL  1719446400           True   
940  CLEMIN20240809D            MIN            CLE  1723161600           True   
941  MILCIN20240830D            CIN            MIL  1724976000           True   
942  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_t

In [23]:
pitchers['earned_run_avg'] = pd.to_numeric(pitchers['earned_run_avg'], errors='coerce')
pitchers['SO_batters_faced'] = pitchers['so'] / pitchers['batters_faced']
pitchers['H_batters_faced'] = pitchers['h'] / pitchers['batters_faced']
pitchers['BB_batters_faced'] = pitchers['bb'] / pitchers['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitchers, df, b_stats, False, 'pitcher')

             game_id home_team_abbr away_team_abbr        date  is_night_game  \
0    NYMSTL20240301D            STL            NYM  1709251200           True   
1    SEACOL20240301D            COL            SEA  1709251200           True   
2     OAKKC20240301D             KC            OAK  1709251200           True   
3    HOUWSH20240301D            WSH            HOU  1709251200           True   
4    CWSCHC20240301D            CHC            CWS  1709251200           True   
..               ...            ...            ...         ...            ...   
938     MIL202405310            MIL            CHC  1717113600          False   
939  ATLCWS20240627D            CWS            ATL  1719446400           True   
940  CLEMIN20240809D            MIN            CLE  1723161600           True   
941  MILCIN20240830D            CIN            MIL  1724976000           True   
942  NYMATL20240926N            ATL            NYM  1727308800           True   

     home_team_win home_pit

In [24]:
print("GAMES:", df['date'])

GAMES: 0      1709251200
1      1709251200
2      1709251200
3      1709251200
4      1709251200
          ...    
938    1717113600
939    1719446400
940    1723161600
941    1724976000
942    1727308800
Name: date, Length: 943, dtype: int64


In [25]:
print(df.columns)

Index(['game_id', 'home_team_abbr', 'away_team_abbr', 'date', 'is_night_game',
       'home_team_win', 'home_pitcher', 'away_pitcher', 'home_trueskill_pre',
       'away_trueskill_pre', 'ts_diff', 'home_team_rest', 'away_team_rest',
       'home_pitcher_rest', 'away_pitcher_rest', 'team_rest_diff',
       'pitcher_rest_diff', 'season', 'month', 'week_num', 'dow', 'dh_game_no',
       'home_batting_batting_avg_10RA', 'home_batting_leverage_index_avg_10RA',
       'home_batting_onbase_perc_10RA',
       'home_batting_onbase_plus_slugging_10RA',
       'away_batting_batting_avg_10RA', 'away_batting_leverage_index_avg_10RA',
       'away_batting_onbase_perc_10RA',
       'away_batting_onbase_plus_slugging_10RA',
       'batting_batting_avg_10RA_diff', 'batting_leverage_index_avg_10RA_diff',
       'batting_onbase_perc_10RA_diff',
       'batting_onbase_plus_slugging_10RA_diff',
       'home_team_pitching_earned_run_avg_10RA',
       'home_team_pitching_SO_batters_faced_10RA',
       'home_

In [26]:
df.shape

(943, 58)

#### Games Stats

In [27]:
df = features.game_stats(games,df)

In [28]:
df.shape

(943, 81)

#### Season Stats

In [29]:
batting_stats = ['a', 'ab', 'bb', 'h', 'pa', 'po', 'r', 'rbi', 'so', 'batting_avg',
             'leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging', 'pitches', 
             're24_bat', 'slugging_perc', 'strikes_total', 'wpa_bat', 'wpa_bat_neg', 
             'wpa_bat_pos']
df = features.add_season_rolling(batting, df, batting_stats, True,'batting')
df.shape

(943, 221)

In [30]:
pitching_stats = ['bb', 'er', 'h', 'hr', 'ip', 'r', 'so', 'batters_faced',
               'earned_run_avg', 'game_score', 'inherited_runners',
               'inherited_score', 'inplay_fb_total', 'inplay_gb_total', 'inplay_ld',
               'inplay_unk', 'leverage_index_avg', 'pitches', 're24_def',
               'strikes_contact', 'strikes_looking', 'strikes_swinging',
               'strikes_total', 'wpa_def','SO_batters_faced','H_batters_faced',
                'BB_batters_faced']
df = features.add_season_rolling(pitching, df, pitching_stats, True,'team_pitching')
df.shape

(943, 410)

In [31]:
if 'away_pitcher' not in df.columns:
    df['away_pitcher'] = 'Unknown'
else:
    df['away_pitcher'].fillna('Unknown', inplace=True)

In [32]:
df = features.add_season_rolling(pitchers, df, pitching_stats, False,'pitcher')
df.shape

(943, 599)

## Cleanup

In [33]:
df = util.fix_na(df, False)

In [34]:
print(df['home_team_win'])

0      0.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
938    0.0
939    0.0
940    0.0
941    0.0
942    0.0
Name: home_team_win, Length: 943, dtype: float64


# Generate Predictions

In [36]:
import pickle
import xgboost as xgb
model = xgb.XGBClassifier()
model.load_model("team_wins_model.json")
filtered_df = df[df['game_id'].isin(test_df['game_id'])]
X_test = filtered_df.drop(columns=[
    'home_team_win', 'home_team_abbr', 'away_team_abbr', 'game_id',
    'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'
])
# Assuming 'model' is your XGBClassifier and 'df' is your DataFrame
# First, get the list of feature names used by the model
model_features = model.get_booster().feature_names

# Ensure the DataFrame only contains the required features and in the correct order
df_for_prediction = X_test[model_features]



In [37]:
filtered_df

,game_id,home_team_abbr,away_team_abbr,date,is_night_game,home_team_win,home_pitcher,away_pitcher,home_trueskill_pre,away_trueskill_pre,...,pitcher_pitches_diff,pitcher_re24_def_diff,pitcher_strikes_contact_diff,pitcher_strikes_looking_diff,pitcher_strikes_swinging_diff,pitcher_strikes_total_diff,pitcher_wpa_def_diff,pitcher_SO_batters_faced_diff,pitcher_H_batters_faced_diff,pitcher_BB_batters_faced_diff
931,TBR202405310,TBR,OAK,1717113600,False,0.0,armstsh01,harriho03,25.000000,24.122696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,SEA202405310,SEA,HOU,1717113600,False,0.0,gilbelo01,arrigsp01,37.259119,23.156763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,NYM202405310,NYM,ARI,1717113600,True,0.0,scottch01,galleza01,19.489830,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,MIN202405310,MIN,KCR,1717113600,False,0.0,paddach01,lynchda02,31.557105,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,BOS202405310,BOS,DET,1717113600,True,0.0,pivetni01,flaheja01,27.784318,28.249349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,LAA202405310,LAA,NYY,1717113600,True,0.0,sandopa02,rodonca01,19.279773,39.937025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,ATL202405310,ATL,WSN,1717113600,True,0.0,kerrra01,willitr01,32.492877,25.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,MIL202405310,MIL,CHC,1717113600,False,0.0,reaco01,taillja01,22.952155,27.895872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
model_features

['is_night_game',
 'home_trueskill_pre',
 'away_trueskill_pre',
 'ts_diff',
 'home_team_rest',
 'away_team_rest',
 'home_pitcher_rest',
 'away_pitcher_rest',
 'team_rest_diff',
 'pitcher_rest_diff',
 'season',
 'month',
 'week_num',
 'dow',
 'dh_game_no',
 'home_batting_batting_avg_10RA',
 'home_batting_leverage_index_avg_10RA',
 'home_batting_onbase_perc_10RA',
 'home_batting_onbase_plus_slugging_10RA',
 'away_batting_batting_avg_10RA',
 'away_batting_leverage_index_avg_10RA',
 'away_batting_onbase_perc_10RA',
 'away_batting_onbase_plus_slugging_10RA',
 'batting_batting_avg_10RA_diff',
 'batting_leverage_index_avg_10RA_diff',
 'batting_onbase_perc_10RA_diff',
 'batting_onbase_plus_slugging_10RA_diff',
 'home_team_pitching_earned_run_avg_10RA',
 'home_team_pitching_SO_batters_faced_10RA',
 'home_team_pitching_H_batters_faced_10RA',
 'home_team_pitching_BB_batters_faced_10RA',
 'away_team_pitching_earned_run_avg_10RA',
 'away_team_pitching_SO_batters_faced_10RA',
 'away_team_pitching_H_

In [38]:
df_for_prediction

,is_night_game,home_trueskill_pre,away_trueskill_pre,ts_diff,home_team_rest,away_team_rest,home_pitcher_rest,away_pitcher_rest,team_rest_diff,pitcher_rest_diff,...,pitcher_pitches_diff,pitcher_re24_def_diff,pitcher_strikes_contact_diff,pitcher_strikes_looking_diff,pitcher_strikes_swinging_diff,pitcher_strikes_total_diff,pitcher_wpa_def_diff,pitcher_SO_batters_faced_diff,pitcher_H_batters_faced_diff,pitcher_BB_batters_faced_diff
931,False,25.000000,24.122696,0.877304,30,24,30,30,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,False,37.259119,23.156763,14.102356,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,True,19.489830,25.000000,-5.510170,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,False,31.557105,25.000000,6.557105,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,True,27.784318,28.249349,-0.465031,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
936,True,19.279773,39.937025,-20.657251,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,True,32.492877,25.000000,7.492877,24,30,30,30,-6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,False,22.952155,27.895872,-4.943716,24,24,30,30,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Now you can use this DataFrame to make predictions
predictions = model.predict(df_for_prediction)
proba = model.predict_proba(df_for_prediction)


In [40]:
predictions

array([0, 0, 0, 1, 0, 0, 0, 0])

In [41]:
proba

array([[0.52170813, 0.47829187],
       [0.518048  , 0.481952  ],
       [0.5098    , 0.49019998],
       [0.46793604, 0.53206396],
       [0.5464802 , 0.4535198 ],
       [0.6066183 , 0.39338169],
       [0.5201719 , 0.47982815],
       [0.5378202 , 0.4621798 ]], dtype=float32)

In [42]:
import pickle
import xgboost as xgb
model = xgb.XGBRegressor()
model.load_model("total_score_model.json")
filtered_df = df[df['game_id'].isin(test_df['game_id'])]
X_test = filtered_df.drop(columns=[
    'home_team_abbr', 'away_team_abbr', 'game_id',
    'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season'
])
# Assuming 'model' is your XGBClassifier and 'df' is your DataFrame
# First, get the list of feature names used by the model
model_features = model.get_booster().feature_names

# Ensure the DataFrame only contains the required features and in the correct order
df_for_prediction = X_test[model_features]


score_predictions = model.predict(df_for_prediction)


In [43]:
score_predictions

array([8.916025, 9.768557, 8.360405, 8.841675, 8.133214, 9.069405,
       9.239534, 8.794895], dtype=float32)

In [44]:
selected_abbr = np.where(np.array(predictions) == 1, test_df['home_team_abbr'], test_df['away_team_abbr'])

# Print the result
print(selected_abbr)

['CHC' 'KCR' 'OAK' 'SEA' 'DET' 'ARI' 'WSN' 'NYY']


In [45]:
import json
json_predictions = []

# Loop through the DataFrame and associated predictions
for index, row in test_df.iterrows():
    # Create a dictionary for each prediction following the specified structure
    prediction = {
        "id": str(index),  # Convert index to string to match 'id: string' type
        "home_team": row['home_team_abbr'],
        "away_team": row['away_team_abbr'],
        "ml_pred": selected_abbr[index],
        "ml_conf": str(max(proba[index])),  # Convert float to string to match 'ml_conf: string'
        "ou_pred": str(score_predictions[index]),  # Placeholder values as specified
        "ou_conf": "0"   # Placeholder values as specified
    }
    json_predictions.append(prediction)

# Convert the list of dictionaries to JSON
json_output = json.dumps(json_predictions, indent=2)
print(json_output)

[
  {
    "id": "0",
    "home_team": "MIL",
    "away_team": "CHC",
    "ml_pred": "CHC",
    "ml_conf": "0.52170813",
    "ou_pred": "8.916025",
    "ou_conf": "0"
  },
  {
    "id": "1",
    "home_team": "MIN",
    "away_team": "KCR",
    "ml_pred": "KCR",
    "ml_conf": "0.518048",
    "ou_pred": "9.768557",
    "ou_conf": "0"
  },
  {
    "id": "2",
    "home_team": "TBR",
    "away_team": "OAK",
    "ml_pred": "OAK",
    "ml_conf": "0.5098",
    "ou_pred": "8.360405",
    "ou_conf": "0"
  },
  {
    "id": "3",
    "home_team": "SEA",
    "away_team": "HOU",
    "ml_pred": "SEA",
    "ml_conf": "0.53206396",
    "ou_pred": "8.841675",
    "ou_conf": "0"
  },
  {
    "id": "4",
    "home_team": "BOS",
    "away_team": "DET",
    "ml_pred": "DET",
    "ml_conf": "0.5464802",
    "ou_pred": "8.133214",
    "ou_conf": "0"
  },
  {
    "id": "5",
    "home_team": "NYM",
    "away_team": "ARI",
    "ml_pred": "ARI",
    "ml_conf": "0.6066183",
    "ou_pred": "9.069405",
    "ou_conf": "

In [39]:
test_df = test_df.sort_values(by=['date','game_id']).reset_index(drop=True)
pred_df = df[df.is_test][['away_pitcher', 'away_team_abbr', 'home_pitcher', 'home_team_abbr']]
pred_df['home'] = pred_df['home_team_abbr']
pred_df['away'] = pred_df['away_team_abbr']
pred_df.drop(columns=['home_team_abbr','away_team_abbr'], inplace=True)
pred_df['xgb_proba']= proba
pred_df['xgb_winner']=pred_df.home
pred_df['xgb_winner'][~pred]=pred_df.away

NameError: name 'proba' is not defined

In [40]:
# get daily odds from covers.com
import requests
from bs4 import BeautifulSoup as bs
html = requests.get('https://www.covers.com/sports/mlb/matchups').text
soup = bs(html)
games = []
for s in soup.findAll('div',{'class':'cmg_matchup_game_box cmg_game_data'}):
    g = {}
    g['home'] = s['data-home-team-shortname-search']
    g['home_odds'] = s['data-game-odd']
    
    if g['home']=='SD':g['home']='SDP'
    if g['home']=='KC':g['home']='KCR'
    if g['home']=='SF':g['home']='SFG'
    if g['home']=='WAS':g['home']='WSN'
    if g['home']=='TB':g['home']='TBR'
    
    games.append(g)
odds = pd.DataFrame(games)

# merge in the odds
pred_df = pd.merge(left=pred_df, right=odds, on='home', how='left')
pred_df['home_odds']=pd.to_numeric(pred_df['home_odds'], errors='coerce')
# pred_df['online_odds'][pred_df.xgb_probability<0.5] = -pred_df['online_odds'] #convert odds to pred winner odds (not home team odds)

In [41]:
# online proba
#https://www.bettingexpert.com/en-au/learn/understanding-betting-odds/how-to-convert-odds
pred_df['online_proba'] = -pred_df['home_odds']/(-pred_df['home_odds']+100)
pred_df['online_proba'][pred_df['home_odds']>0] = 100/(pred_df['home_odds']+100)

# Confidence
pred_df['confidence'] = np.abs(pred_df['xgb_proba']-0.5)+.5
online_conf = np.abs(pred_df['online_proba']-0.5)+.5

pred_df['conf_diff'] = pred_df['confidence'] - online_conf
pred_df['conf_diff'][(pred_df['xgb_proba']>.5)&(pred_df['online_proba']<.5)] = 'Contrary'
pred_df['conf_diff'][(pred_df['xgb_proba']<.5)&(pred_df['online_proba']>.5)] = 'Contrary'

KeyError: 'xgb_proba'

In [ ]:
# merge in team names
teams = pd.read_csv("../data/teams.csv")
pred_df = pd.merge(left=pred_df, right=teams, 
                   left_on='xgb_winner',right_on='Abbr',
                   how='left')

pred_df['pred_winner'] = pred_df['Team']
pred_df.drop(columns=['xgb_winner','Abbr','Team'], inplace=True)

In [ ]:
from IPython.display import HTML
pd.options.display.float_format = '{:.3f}'.format

pred_df['conf'] = pred_df.confidence
HTML(pred_df.sort_values(by='confidence', ascending=False).to_html(index=False))

In [ ]:
print("2-Team Parlays:")
pred_df = pred_df.sort_values(by='conf', ascending=False).reset_index(drop=True)
for i in range(5):
    t1 = pred_df.iloc[i]
    t2 = pred_df.iloc[i+1]
    print(f"- {t1.pred_winner}, {t2.pred_winner}  \t{t1.conf*t2.conf: .3f}")